Step 1: Download Kafka

In [ ]:
!wget https://archive.apache.org/dist/kafka/2.8.0/kafka_2.12-2.8.0.tgz

Step 2: Extract Kafka

In [ ]:
!tar -xzf kafka_2.12-2.8.0.tgz

Step 3: Start a MySQL Database server

In [ ]:
!start_mysql

Step 4: Connect to the MySQL Database

In [ ]:
!mysql --host=127.0.0.1 --port=3306 --user=root --password=NDE2Mi1uMm9tYW5v

Step 5: Create a database named tolldata

In [ ]:
mysql> create database tolldata;

Step 6: Create a table named livetolldata

In [ ]:
mysql> use tolldata;

mysql> create table livetolldata(timestamp datetime,vehicle_id int,vehicle_type char(15),toll_plaza_id smallint);

Step 7: Disconnect from MySQL server

In [ ]:
mysql> exit

Step 8: Install Kafka Python driver

In [ ]:
!python3 -m pip install kafka-python

Step 9: Install the MySQL Python driver

In [ ]:
!python3 -m pip install mysql-connector-python==8.0.31

Step 10: Start Zookeeper

In [ ]:
!cd kafka_2.12-2.8.0
!bin/zookeeper-server-start.sh config/zookeeper.properties

Step 11: Start Kafka server

In [ ]:
!cd kafka_2.12-2.8.0
!bin/kafka-server-start.sh config/server.properties

Step 12: Create a topic named toll in kafka

In [ ]:
!cd kafka_2.12-2.8.0
!bin/kafka-topics.sh --create --topic toll --bootstrap-server localhost:9092

Step 13: Download toll traffic simulator program

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0250EN-SkillsNetwork/labs/Final%20Assignment/toll_traffic_generator.py

Step 14: Customize the generator program to steam to toll topic

In [ ]:
"""
Top Traffic Simulator
"""
from time import sleep, time, ctime
from random import random, randint, choice
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='localhost:9092')

TOPIC = 'toll'

VEHICLE_TYPES = ("car", "car", "car", "car", "car", "car", "car", "car",
                 "car", "car", "car", "truck", "truck", "truck",
                 "truck", "van", "van")
for _ in range(100000):
    vehicle_id = randint(10000, 10000000)
    vehicle_type = choice(VEHICLE_TYPES)
    now = ctime(time())
    plaza_id = randint(4000, 4010)
    message = f"{now},{vehicle_id},{vehicle_type},{plaza_id}"
    message = bytearray(message.encode("utf-8"))
    print(f"A {vehicle_type} has passed by the toll plaza {plaza_id} at {now}.")
    producer.send(TOPIC, message)
    sleep(random() * 2)


: 

Step 15: Run the Toll Traffic Simulator

In [ ]:
!python3 toll_traffic_generator.py

Step 16: Download streaming data reader (consumer)

In [ ]:
!https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0250EN-SkillsNetwork/labs/Final%20Assignment/streaming_data_reader.py

Step 17: Customize the consumer program to write into a MySQL database table

In [ ]:
"""
Streaming data consumer
"""
from datetime import datetime
from kafka import KafkaConsumer
import mysql.connector

TOPIC='toll'
DATABASE = 'tolldata'
USERNAME = 'root'
PASSWORD = 'NDE2Mi1uMm9tYW5v'

print("Connecting to the database")
try:
    connection = mysql.connector.connect(host='localhost', database=DATABASE, user=USERNAME, password=PASSWORD)
except Exception:
    print("Could not connect to database. Please check credentials")
else:
    print("Connected to database")
cursor = connection.cursor()

print("Connecting to Kafka")
consumer = KafkaConsumer(TOPIC)
print("Connected to Kafka")
print(f"Reading messages from the topic {TOPIC}")
for msg in consumer:

    # Extract information from kafka

    message = msg.value.decode("utf-8")

    # Transform the date format to suit the database schema
    (timestamp, vehcile_id, vehicle_type, plaza_id) = message.split(",")

    dateobj = datetime.strptime(timestamp, '%a %b %d %H:%M:%S %Y')
    timestamp = dateobj.strftime("%Y-%m-%d %H:%M:%S")

    # Loading data into the database table

    sql = "insert into livetolldata values(%s,%s,%s,%s)"
    result = cursor.execute(sql, (timestamp, vehcile_id, vehicle_type, plaza_id))
    print(f"A {vehicle_type} was inserted into the database")
    connection.commit()
connection.close()


Step 17: Verify that streamed data is being collected in the database table